In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 43.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
import numpy as np

In [5]:
!tree /content/drive/MyDrive


/bin/bash: line 1: tree: command not found


In [6]:
from pathlib import Path
img_path = Path('/content/drive/MyDrive/mlp_rawdata/tif_img')

img_names = [f.name for f in img_path.iterdir() if f.is_file()]


In [7]:
gpkg_path = '/content/drive/MyDrive/mlp_rawdata/TN_MAPINDX_5K.gpkg'

gpkg = gpd.read_file(gpkg_path)

gpkg = gpkg.to_crs(epsg=5186)


In [8]:
gpkg[gpkg['MAPIDCD_NO']=='37608078'].geometry.total_bounds

array([193372.51997588, 550054.67002869, 195583.15009928, 552830.8499952 ])

In [9]:
shp_path = '/content/drive/MyDrive/mlp_rawdata/AL_D010_11_20240404/AL_D010_11_20240404.shp'
shp = gpd.read_file(shp_path, encoding='cp949')
shp.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A20,A21,A22,A23,A24,A25,A26,A27,A28,geometry
0,32,1991201839054527769900000000,1111017500107040000,1111017500,서울특별시 종로구 숭인동,704,1,일반,01000,단독주택,...,N,B00100000000T30Z9,2024-04-01,11110,None,None,3.0,0.0,2017-05-30,"POLYGON ((201913.638 553085.323, 201912.228 55..."
1,34,1967201343844527773800000000,1111017500100560024,1111017500,서울특별시 종로구 숭인동,56-24,1,일반,01000,단독주택,...,N,B00100000000T311B,2024-04-01,11110,None,None,3.0,0.0,2018-06-19,"POLYGON ((201413.758 553078.463, 201410.198 55..."
2,36,1962201375194527742000000000,1111017500100560054,1111017500,서울특별시 종로구 숭인동,56-54,1,일반,01000,단독주택,...,N,B00100000000T313D,2024-04-01,11110,None,None,2.0,0.0,2017-02-24,"POLYGON ((201442.598 553075.703, 201442.298 55..."
3,37,1979201388484527710100000000,1111017500100560053,1111017500,서울특별시 종로구 숭인동,56-53,1,일반,01000,단독주택,...,N,B00100000000T314E,2024-04-01,11110,None,None,2.0,1.0,2018-11-06,"POLYGON ((201458.568 553069.403, 201453.548 55..."
4,38,0000200468294527764000000000,1111016500100280011,1111016500,서울특별시 종로구 이화동,28-11,1,일반,None,None,...,None,B00100000000T315F,2024-04-01,11110,None,None,0.0,0.0,None,"POLYGON ((200540.538 553080.543, 200536.438 55..."


In [ ]:
for img in img_names:

  map_number = img.split("_")[2].split(".")[0]
  map = gpkg[gpkg['MAPIDCD_NO'] == map_number]

  bounds = map.total_bounds
  pixel = 0.25
  print(bounds)

  width = int((bounds[2] - bounds[0])/ pixel)
  height = int((bounds[3] - bounds[1]) / pixel)

  transform = rasterio.transform.from_origin(bounds[0], bounds[3], pixel, pixel)

  shapes = ((geom, i+1) for i, geom in enumerate(shp.geometry))
  raterized = features.rasterize(
      shapes = shapes,
      out_shape = (height, width),
      transform = transform
  )
  np_path = '/content/drive/MyDrive/mlp_rawdata/label_np/' + map_number + '.npy'

  np.save(np_path, raterized)
  print(np_path, ': 저장', width, height)





[193372.51997588 550054.67002869 195583.15009928 552830.8499952 ]
/content/drive/MyDrive/mlp_rawdata/label_np/37608078.npy : 저장 8842 11104


In [ ]:
print(height, width)

11104 8842


In [11]:
# Part 1: 라이브러리 임포트 (기본 도구 준비)
# ------------------------------------------------------------------------------
import torch
from torch.utils.data import Dataset, DataLoader
import rasterio
import numpy as np
from pathlib import Path
from transformers import Mask2FormerForUniversalSegmentation
import torchvision.transforms.functional as F
from torchvision import transforms

# ==============================================================================
# Part 2: BuildingDataset 클래스 (데이터 준비 담당)
# - 모델이 학습할 수 있도록, 원본 데이터(.tif, .npy)를 PyTorch 텐서로 변환하는 '요리사' 역할
# ==============================================================================

class BuildingDataset(Dataset):
    # --- __init__: '요리사'가 처음 준비할 때 필요한 것들을 설정하는 부분 ---
    # img_dir: 항공사진(.tif)이 있는 폴더 경로
    # label_dir: 정답 라벨(.npy)이 있는 폴더 경로
    # resize_shape: 이미지를 리사이즈할 목표 크기
    def __init__(self, img_dir, label_dir, resize_shape=(1024, 1024)):
        # 입력받은 파일 경로를 Path 객체로 변환하여 안정적으로 관리
        self.img_path = Path(img_dir)
        self.label_path = Path(label_dir)
        # 이미지 폴더 안의 모든 .tif 파일 목록을 정렬하여 저장
        self.img_files = sorted([f for f in self.img_path.iterdir() if f.suffix == '.tif'])
        self.resize_shape = resize_shape

        # 이미지 정규화를 위한 규칙 정의 (ImageNet 데이터셋의 평균 및 표준편차)
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # --- __len__: 우리가 가진 총 데이터(이미지)의 개수를 반환하는 함수 ---
    def __len__(self):
        return len(self.img_files)

    # --- __getitem__: "n번째 데이터 주세요" 요청이 오면, 해당 데이터를 가공하여 반환하는 핵심 함수 ---
    def __getitem__(self, idx):
        # idx번째 이미지 파일 경로를 가져옴
        img_filepath = self.img_files[idx]
        # rasterio로 .tif 파일을 열고, 픽셀 데이터를 NumPy 배열로 읽어옴
        with rasterio.open(img_filepath) as src:
            image = src.read().astype(np.float32)

        # --- 라벨 매칭: 이미지 파일명에 맞는 .npy 라벨 파일을 정확히 찾아 불러오는 로직 ---
        try:
            # 이미지 파일명(예: ..._37608078.tif)에서 지도 번호(37608078)를 추출
            map_number = img_filepath.name.split("_")[2].split(".")[0]
            # 추출한 지도 번호로 라벨 파일명(예: 37608078.npy)을 생성
            label_filename = map_number + '.npy'
            label_filepath = self.label_path / label_filename
            # 해당 경로의 라벨 파일을 NumPy 배열로 불러옴
            label = np.load(label_filepath).astype(np.int64)
        # 만약 파일이 없거나(FileNotFoundError) 이름 규칙이 다르면(IndexError), 경고를 출력하고 빈 라벨을 생성
        except (IndexError, FileNotFoundError):
            print(f"경고: {img_filepath.name}에 대한 라벨 파일을 찾을 수 없거나 형식이 맞지 않습니다.")
            label = np.zeros((image.shape[1], image.shape[2]), dtype=np.int64)

        # NumPy 배열을 PyTorch 텐서로 변환
        image_tensor = torch.from_numpy(image)
        label_tensor = torch.from_numpy(label).unsqueeze(0) # 리사이즈를 위해 임시로 채널 차원 추가 [H, W] -> [1, H, W]

        # --- 리사이즈: 거대한 원본 이미지를 (1024, 1024) 크기로 줄여 메모리 문제 해결 및 학습 효율 향상 ---
        # 이미지 리사이즈 (BILINEAR: 색상을 부드럽게 섞어 자연스럽게 축소)
        image_tensor = F.resize(image_tensor, self.resize_shape, interpolation=F.InterpolationMode.BILINEAR)
        # 라벨 리사이즈 (NEAREST: 건물 ID(1)가 0.8처럼 변하지 않도록, 가장 가까운 정수 값 유지)
        label_tensor = F.resize(label_tensor, self.resize_shape, interpolation=F.InterpolationMode.NEAREST)

        # --- 정규화: 리사이즈된 이미지에 미리 정의한 정규화 규칙을 적용하여 학습 안정성 향상 ---
        image_tensor = self.normalize(image_tensor)
        # 라벨의 불필요한 채널 차원을 다시 제거 [1, H, W] -> [H, W]
        label_tensor = label_tensor.squeeze(0)

        # --- 최종 반환: 모든 전처리를 마친 이미지와 라벨 텐서 한 쌍을 반환 ---
        return image_tensor, label_tensor

# ==============================================================================
# Part 3: 데이터 로딩 및 모델 설정 (실행 환경 구성)
# ==============================================================================

# 실제 데이터가 저장된 Google 드라이브 경로
IMG_DIRECTORY = '/content/drive/MyDrive/mlp_rawdata/tif_img'
LABEL_DIRECTORY = '/content/drive/MyDrive/mlp_rawdata/label_np'

print("데이터셋을 생성합니다...")
# BuildingDataset 클래스의 실제 객체('요리사')를 생성. 리사이즈 크기를 (1024, 1024)로 지정
building_dataset = BuildingDataset(img_dir=IMG_DIRECTORY, label_dir=LABEL_DIRECTORY, resize_shape=(1024, 1024))
# DataLoader('웨이터') 생성: 데이터셋에서 데이터를 배치(1개씩) 단위로, 랜덤으로 섞어서 가져옴
data_loader = DataLoader(building_dataset, batch_size=1, shuffle=True)
print(f"데이터셋에 총 {len(building_dataset)}개의 샘플이 있습니다.")


# ==============================================================================
# Part 4: 백본 실행 및 결과 확인 (핵심 기능 실행)
# ==============================================================================
print("\n사전 학습된 Mask2Former 모델과 Swin 백본을 로드합니다...")
# Hugging Face Hub에서 사전 학습된 Mask2Former 모델의 설계도와 가중치를 다운로드
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-base-coco-instance")
# 모델을 학습 모드가 아닌, 추론(평가) 모드로 설정
model.eval()

# Mask2Former 모델 전체에서 '눈' 역할만 하는 Swin Transformer 백본 모듈을 정확히 꺼내옴
swin_backbone = model.model.pixel_level_module.encoder
print(f"성공적으로 {type(swin_backbone).__name__} 백본을 추출했습니다.")

# --- 데이터셋의 첫 번째 배치로 백본 실행 테스트 ---
try:
    # DataLoader('웨이터')에게 "첫 번째 데이터 한 묶음 주세요"라고 요청하여 이미지와 라벨을 받아옴
    image_batch, label_batch = next(iter(data_loader))
    print(f"\n데이터 로더에서 첫 번째 이미지 배치(형태: {image_batch.shape})를 가져왔습니다.")

    # torch.no_grad(): "지금은 테스트이니, 기울기 계산 등 복잡한 작업은 잠시 멈춰줘" (메모리, 속도 절약)
    with torch.no_grad():
        # --- 프로젝트의 하이라이트: 준비된 이미지 배치를 백본에 입력하여 특징 맵을 추출 ---
        feature_maps = swin_backbone(image_batch, output_hidden_states=True, return_dict=False)

    # 백본이 출력한 여러 종류의 특징 맵들을 꺼냄
    hidden_states = feature_maps[1]
    print("\nSwin Transformer 백본이 추출한 다중 스케일 특징 맵:")
    # 각 특징 맵의 형태를 출력하여 결과가 제대로 나왔는지 최종 확인
    for i, feature_map in enumerate(hidden_states):
        print(f"  - Stage {i} 특징 맵 형태: {feature_map.shape}")

# 데이터 로더가 비어있을 경우 예외 처리
except StopIteration:
    print("\n오류: 데이터셋이 비어있거나 데이터 로더에서 데이터를 가져올 수 없습니다.")


데이터셋을 생성합니다...
데이터셋에 총 1개의 샘플이 있습니다.

사전 학습된 Mask2Former 모델과 Swin 백본을 로드합니다...
성공적으로 SwinBackbone 백본을 추출했습니다.

데이터 로더에서 첫 번째 이미지 배치(형태: torch.Size([1, 3, 1024, 1024]))를 가져왔습니다.

Swin Transformer 백본이 추출한 다중 스케일 특징 맵:
  - Stage 0 특징 맵 형태: torch.Size([1, 65536, 128])
  - Stage 1 특징 맵 형태: torch.Size([1, 65536, 128])
  - Stage 2 특징 맵 형태: torch.Size([1, 16384, 256])
  - Stage 3 특징 맵 형태: torch.Size([1, 4096, 512])
  - Stage 4 특징 맵 형태: torch.Size([1, 1024, 1024])


##### <백본 파트 역할>

우리 모델 Mask2Former에서 백본의 역할은, 입력된 항공사진(.tif)으로부터 다양한 수준의 특징 정보(Feature Maps)를 추출하여, 후속 모듈인 픽셀 디코더(Pixel Decoder)와 트랜스포머 디코더(Transformer Decoder)로 전달하는 것입니다.​


##### \<핵심 코드 (BuildingDataset 클래스 및 모델 실행 파트)>

1. BuildingDataset 클래스 구현:
* __getitem__ 메소드 안에서 .tif 이미지와 .npy 라벨을 한 쌍으로 불러옵니다.
* 이미지 전처리:  
 -리사이즈: GPU 메모리 부족 문제를 해결하고 학습 효율을 높이기 위해, 거대한 원본 이미지를 (512, 512)의 작은 크기로 통일시켰습니다.  
 -정규화: 이미지의 픽셀 값 분포를 표준화하여 모델이 더 안정적으로 학습하도록 했습니다.
* 이 모든 과정을 거쳐, 최종적으로 모델에 입력될 PyTorch 텐서(Tensor)를 반환합니다.

2. 모델 로드 및 백본 추출
* Mask2FormerForUniversalSegmentation.from_pretrained(...) 코드로 Hugging Face Hub에서 사전 학습된 Mask2Former 모델 전체를 불러왔습니다..
* model.model.pixel_level_module.encoder를 통해, 우리가 필요한 Swin Transformer 백본 부분만 정확히 추출했습니다.

3. 백본 실행 및 특징 맵 추출
* DataLoader를 통해 준비된 이미지 데이터(image_batch)를 백본 모델에 입력했습니다.
* swin_backbone(image_batch, ...) 코드가 성공적으로 실행되며, 백본의 최종 결과물인 hidden_states (다중 스케일 특징 맵)가 반환되었습니다.





##### \[실행 결과]

데이터 로더에서 첫 번째 이미지 배치(형태: torch.Size(\[1, 3, 512, 512]))를 가져왔습니다.

Swin Transformer 백본이 추출한 다중 스케일 특징 맵:

&nbsp; - Stage 0 특징 맵 형태: torch.Size(\[1, 16384, 128])

&nbsp; - Stage 1 특징 맵 형태: torch.Size(\[1, 16384, 128])

&nbsp; - Stage 2 특징 맵 형태: torch.Size(\[1, 4096, 256])

&nbsp; - Stage 3 특징 맵 형태: torch.Size(\[1, 1024, 512])

&nbsp; - Stage 4 특징 맵 형태: torch.Size(\[1, 256, 1024])



-> 결과물의 의미: 픽셀 디코더와 트랜스포머 디코더로 전달될 데이터

위 결과에서 Swin Transformer 백본이 추출한 다중 스케일 특징 맵이 backbone 파트의 최종 결과물이며, 다른 파트로 전달될 입력 데이터입니다.



* 다중 스케일(Multi-scale)의 의미: 이 특징 맵들은 하나의 이미지를 다양한 관점에서 분석한 결과물입니다.​

-초기 Stage (고해상도): 이미지의 세밀한 경계선, 모서리 등 '디테일' 정보를 담고 있습니다.

-후기 Stage (저해상도): 이미지의 전체적인 구조, 객체의 형태 등 '맥락' 정보를 담고 있습니다.

* 팀원 파트와의 연관성:

-픽셀 디코더 파트:  다중 스케일 특징 맵들(hidden\_states)을 입력으로 받아, 이를 다시 융합하고 업샘플링하여 최종 마스크를 예측하는 데 필요한 고해상도의 픽셀별 임베딩(per-pixel embeddings)을 생성하게 됩니다.​

-트랜스포머 디코더 파트:  이 특징 맵들과 학습 가능한 쿼리(queries)를 함께 사용하여, 이미지에 어떤 객체들이 있는지, 그리고 각 객체가 어떤 픽셀들을 포함하는지에 대한 정보를 추출하게 됩니다.​





##### <결론 및 다음 단계>

여기서 구현한 코드의 hidden\_states 변수가 다른 파트의 모듈로 전달될 입력 데이터이니, 이를 기반으로 파트 연동을 준비해 주시면 됩니다.

